# Laboratoire 3 : Machines à vecteurs de support et réseaux neuronaux
#### Département du génie logiciel et des technologies de l’information

| Étudiants             | LEMARCHANT HUGO - LEMH03039705 * TAN ELODIE - TANE25619607 * JACQUES-SYLVAIN LECOINTRE LECJ19128301|
|-----------------------|---------------------------------------------------------|
| Cours                 | GTI770 - Systèmes intelligents et apprentissage machine |
| Session               | Automne 2018                                            |
| Groupe                | C                                                       |
| Numéro du laboratoire | 02                                                      |
| Professeur            | Prof. Hervé Lombaert                                    |
| Chargé de laboratoire | Pierre-Luc Delisle                                      |
| Date                  | 31/10/2018                                              |

In [1]:
import numpy as np
import pandas as pd
import pylab as pl

from sklearn.model_selection import train_test_split, cross_val_score , GridSearchCV , StratifiedShuffleSplit
from sklearn.svm import SVC

In [2]:
feature_vectors = pd.read_csv('galaxy_feature_vectors.csv', delimiter=',', header=None)

In [3]:
X_galaxy = pd.read_csv('galaxy_feature_vectors.csv', delimiter=',', header=None).values[:, 0:-1]
Y_galaxy = pd.read_csv('galaxy_feature_vectors.csv', delimiter=',', header=None).values[:, -1:].astype(int).flatten()

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X_galaxy, Y_galaxy, test_size=0.20, random_state=42,stratify=Y_galaxy)

### Machines à vecteurs de support 

In [14]:
jobs = 6
cache_size=2048
k=5

cv = StratifiedShuffleSplit(n_splits=k, test_size=0.2, random_state=42)
svc = SVC(cache_size=cache_size)

#### Recherche par grille : Lineaire

In [ ]:
param_grid_linear = {'kernel': ['linear'], 'C': [10 ** (-3), 10 ** (-1), 1, 10], 'class_weight': ['balanced'],'gamma':['scale']}
grid_linear = GridSearchCV(svc, param_grid=param_grid_linear, cv=cv, n_jobs=jobs, scoring='accuracy',verbose=4)

In [ ]:
grid_linear.fit(X_train, Y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


In [ ]:
print("LINEAR : The best hyperparameters are %s with a score of %0.2f"% (grid_linear.best_params_, grid_linear.best_score_))

##### Recherche par grille : RBF

In [ ]:
param_grid_rbf = {'kernel':('rbf'),'C':[10**(-3), 10**(-1), 1, 10],'gamma':[10**(-3), 10**(-1), 1, 10]}
grid_rbf = GridSearchCV(svc, param_grid=param_grid_rbf, cv=cv, n_jobs=jobs, scoring='accuracy',verbose=4)

In [9]:
grid_rbf.fit(X_train, Y_train)

In [ ]:
print("RBF : The best hyperparameters are %s with a score of %0.2f" % (grid_rbf.best_params_, grid_rbf.best_score_))

In [ ]:
C_range=param_grid_rbf['C']
gamma_range=param_grid_rbf['gamma']

grid=grid_rbf
score_dict = grid.grid_scores_

# We extract just the scores
scores = [x[1] for x in score_dict]
scores = np.array(scores).reshape(len(C_range), len(gamma_range))

# Make a nice figure
pl.figure(figsize=(8, 6))
pl.subplots_adjust(left=0.15, right=0.95, bottom=0.15, top=0.95)
pl.imshow(scores, interpolation='nearest', cmap=pl.cm.spectral)
pl.xlabel('gamma')
pl.ylabel('C')
pl.colorbar()
pl.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
pl.yticks(np.arange(len(C_range)), C_range)
pl.show()

## Code Bonus

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2
import csv
matplotlib.use("Agg")
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Dropout
from keras import backend as K
from tensorflow.python.client import device_lib
from keras.models import load_model
print(device_lib.list_local_devices())

class NetCNN:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model
        model = Sequential()
        inputShape = (height, width, depth)
 
        # if we are using "channels first", update the input shape
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
        # first set of CONV => RELU => POOL layers
        model.add(Conv2D(20, (5, 5), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(Dropout(0.2))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # second set of CONV => RELU => POOL layers
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(Dropout(0.2))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # third set of CONV => RELU => POOL layers
        model.add(Conv2D(20, (5, 5), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(Dropout(0.2))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
        model.add(Dropout(0.3))
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model
    
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []

with open("..\\data\\data\\csv\\galaxy\\galaxy_label_data_set.csv", 'rt') as csvFile:
    reader = csv.reader(csvFile, delimiter=",")
    # loop over the input images
    for index, row in enumerate(reader):
        if index in range(1,1000):
            # load the image, pre-process it, and store it in the data list
            file = "..\\data\\data\\images\\"+row[0]+".jpg"
            image = cv2.imread(file)
            image = cv2.resize(image, (212, 212))
            image = img_to_array(image)
            data.append(image)
            label = (0 if row[1] == 'smooth' else 1)
            labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,height_shift_range=0.1, 
      shear_range=0.2, zoom_range=0.2,horizontal_flip=True, fill_mode="nearest")

EPOCHS = 50
INIT_LR = 1e-3
BS = 25

# initialize the model
print("[INFO] compiling model...")
model = NetCNN.build(width=212, height=212, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

# save the model to disk
print("[INFO] serializing network...")
model.save("..\\data\\tp3\\model.hdf5")

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Smooth/Spiral")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("Graphs\\3c25bs3lr.png")

## Introduction

## Question 1 
### Présentation de la méthode de validation

## Question 2
### Description du modèle élaboré

## Question 3
### Analyse de la phase d'entraînement

## Question 4
### Matrice des expérimentations

### Étude des hyperparamètres des modèles

## Question 5
### Matrice des expérimentations

### Études des hyperparamètres des modèles

## Question 6
### Discussion de l'impact de la taille des données

## Question 7
### Discussion de l'impact du bruit

## Question 8
### Formulation des recommandations

## Question 9
### Améliorations possibles

## Question Bonus

Le réseau de neurones convolutif est un réseau de neurones classique à la différence que les neurones de la couche de convolution partagent le même poids, qui est en fait une fenêtre de convolution. C’est à dire dans notre cas une matrice 5x5 de poids. De cette manière chaque neurone correspond à un déplacement de la matrice sur l’image (un déplacement de la fenêtre de convolution). L’objectif pour ces couches est de trouver les poids de cette matrice tels qu’ils minimisent la fonction de coût (ici binary_crossentropy). Ensuite après cette couche de convolution, nous faisons un maxPooling qui revient à considérer un cadre de 2x2 neurones et de synthétiser cette fenêtre en la valeur maximale présente dans cette fenêtre (on utilise le max plutôt que la moyenne car la littérature montre que le maximum décrit mieux la fenêtre que la moyenne).

Nos de convolutions suivies de maxPooling activées avec ReLU, une couche dense (ou fully-connected) avec la même fonction d’activation ReLU et enfin une couche de décision de deux neurones avec une activation softMax pour traduire les résultats en probabilités.

Nous avons de plus ajouté un dropout sur nos différentes couches (20% et 30% pour les couches de convolution et la couche dense) pour empêcher notre réseau de faire du sur-apprentissage. En effet avec ces extinction de neurones, on entraîne virtuellement un forêt de réseaux tels que le réseau complet une fois entraîné sera la décision moyenne de cette forêt.
Pour ce classifieur, nous avions accès à trois hyper-paramètres : le taux d’apprentissage, la taille du mini batch et le nombre de couches de convolutions.

Pour la taille du mini batch, nous ne pouvons pas faire varier énormément car si celle ci devient trop importante le batch ne peut plus être chargé en mémoire. Nous avons donc testé avec des tailles de 25 et 10. Le taux d'apprentissage initial a été paramétré à 1e-1, 1e-3 et 1e-5. Nous avons comparé d’un autre côté l’apprentissage via 2 et 3 couches de convolution.

Ci dessous la comparaison entre les deux tailles de mini batch:

__Batch size = 10__
!["Batch size = 10"](Graphs/2c10bs3lr.png)
__Batch size = 25__
!["Batch size = 25"](Graphs/2c25bs3lr.png)

Ci dessous la comparaison entre deux et trois couches convolutives:

__3 couches de convolution__
!["3 couches de convolution"](Graphs/3c25bs3lr.png)
__2 couches de convolution__
!["2 couches de convolution"](Graphs/2c25bs3lr.png)

Ci dessous la comparaison entre 3 valeurs de learning rate:

__Learning rate = 1e-1__
!["Learning rate = 1e-1"](Graphs/2c25bs1lr.png)
__Learning rate = 1e-5__
!["Learning rate = 1e-5"](Graphs/2c25bs5lr.png)
__Learning rate = 1e-3__
!["Learning rate = 1e-3"](Graphs/2c25bs3lr.png)

On observe que le batch size de 10 permet un apprentissage plus constant, les écarts entre validation et entraînement sont moins importants. On observe aussi des meilleurs résultats sur les 10 premières itérations. Les mêmes effets sont visibles à l’ajout d’une troisième couche de convolution.
Le learning rate lui à des effets bien plus visibles, un learning rate très grand fait diverger instantanément le réseau et le gradient devient très grand. Le learning rate de 1e-5 lui donne une courbe très lisse avec une précision égale au terme des 50 itérations.

## Conclusion

## Bibliographie